In [2]:
pwd

'/home/wsuser/work'

In [3]:
from keras.preprocessing.image import ImageDataGenerator 
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten

### Image Preprocessing and Data Augmentation

In [4]:
train_data=ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)

In [5]:
test_data =ImageDataGenerator (rescale = 1)

In [6]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='30yrk4b8T3T_Ilng13DxclWWHGAJ6Pwzb0kMj_q2Onsf',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'fertilizerrecommendation-donotdelete-pr-z7k2n4vylc34ok'
object_key = 'Dataset Plant Disease.zip'

streaming_body_1 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/

if not hasattr(streaming_body_1,"__iter__"):
    streaming_body_1.__iter__=types.MethodType(__iter__,streaming_body_1)


In [7]:
from io import BytesIO
import zipfile
unzip=zipfile.ZipFile(BytesIO(streaming_body_1.read()),'r')
file_path=unzip.namelist()
for path in file_path:
    unzip.extract(path)

In [8]:
import os
filenames=os.listdir('/home/wsuser/work/Dataset Plant Disease/fruit-dataset')

In [9]:
X_train=train_data.flow_from_directory("/home/wsuser/work/Dataset Plant Disease/fruit-dataset/train",target_size=(128,128),batch_size=32,class_mode="categorical")

Found 5384 images belonging to 6 classes.


In [10]:
X_test=test_data.flow_from_directory("/home/wsuser/work/Dataset Plant Disease/fruit-dataset/test",target_size=(128,128),batch_size=32,class_mode="categorical")

Found 1686 images belonging to 6 classes.


In [11]:
X_train.class_indices.keys()

dict_keys(['Apple___Black_rot', 'Apple___healthy', 'Corn_(maize)___Northern_Leaf_Blight', 'Corn_(maize)___healthy', 'Peach___Bacterial_spot', 'Peach___healthy'])

In [12]:
X_test.class_indices.keys()

dict_keys(['Apple___Black_rot', 'Apple___healthy', 'Corn_(maize)___Northern_Leaf_Blight', 'Corn_(maize)___healthy', 'Peach___Bacterial_spot', 'Peach___healthy'])

### Initializing The Model


In [13]:
model=Sequential()

### Add CNN Layers

In [14]:
model.add(Convolution2D(32,(3,3),input_shape=(128,128,3),activation='relu'))

In [15]:
#pooling layer
model.add(MaxPooling2D(pool_size=(2,2)))

In [16]:
#flatten layer
model.add(Flatten())

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 63, 32)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 127008)            0         
                                                                 
Total params: 896
Trainable params: 896
Non-trainable params: 0
_________________________________________________________________


### Add Dense Layers

In [18]:
#dense layers
model.add(Dense(300,kernel_initializer='uniform', activation = 'relu'))
model.add(Dense(250,kernel_initializer = 'random_uniform', activation = 'relu'))
model.add(Dense(6,kernel_initializer = 'random_uniform', activation = 'softmax')) #output layer

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 63, 32)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 127008)            0         
                                                                 
 dense (Dense)               (None, 300)               38102700  
                                                                 
 dense_1 (Dense)             (None, 250)               75250     
                                                                 
 dense_2 (Dense)             (None, 6)                 1506      
                                                        

In [20]:
#compile model
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=["accuracy"])

In [21]:
len(X_train)
len(X_test)

53

In [22]:
model.fit_generator(X_train,steps_per_epoch=len(X_train),epochs=10,validation_data=X_test,validation_steps=len(X_test))

/tmp/wsuser/ipykernel_164/2793458004.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(X_train,steps_per_epoch=len(X_train),epochs=10,validation_data=X_test,validation_steps=len(X_test))


Epoch 1/10
169/169 [==============================] - 143s 843ms/step - loss: 0.7257 - accuracy: 0.7396 - val_loss: 108.9052 - val_accuracy: 0.7894
Epoch 2/10
169/169 [==============================] - 143s 849ms/step - loss: 0.2952 - accuracy: 0.8977 - val_loss: 344.9380 - val_accuracy: 0.7005
Epoch 3/10
169/169 [==============================] - 140s 828ms/step - loss: 0.2384 - accuracy: 0.9179 - val_loss: 699.4307 - val_accuracy: 0.6590
Epoch 4/10
169/169 [==============================] - 146s 865ms/step - loss: 0.1758 - accuracy: 0.9382 - val_loss: 779.2010 - val_accuracy: 0.6518
Epoch 5/10
169/169 [==============================] - 147s 866ms/step - loss: 0.1622 - accuracy: 0.9435 - val_loss: 911.0378 - val_accuracy: 0.6139
Epoch 6/10
169/169 [==============================] - 142s 840ms/step - loss: 0.1335 - accuracy: 0.9549 - val_loss: 1377.7058 - val_accuracy: 0.5071
Epoch 7/10
169/169 [==============================] - 146s 861ms/step - loss: 0.1020 - accuracy: 0.9653 - val_l

In [23]:
model.save("fruit_training.h5")

In [24]:
!tar -zcvf fruit-model_new.tgz fruit_training.h5

fruit_training.h5


In [25]:
ls -1

'Dataset Plant Disease'/
fruit-model_new.tgz
fruit_training.h5


In [26]:
!pip install watson-machine-learning-client --upgrade

     |████████████████████████████████| 538 kB 12.6 MB/s eta 0:00:01


In [27]:
from ibm_watson_machine_learning import APIClient
wml_credentials={
    "url":"https://us-south.ml.cloud.ibm.com",
    "apikey":"RwEM3L4sWwvYmnY6F6AnNLUdsi3kiZaeglIqI3sNeAm8"
}
client=APIClient(wml_credentials)

In [28]:
client=APIClient(wml_credentials)

In [29]:
def guid_from_space(client,space_name):
    space=client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']["name"]==space_name)['metadata']['id'])


In [30]:
space_uid=guid_from_space(client,'fruit-model-space')
print(space_uid)

9b90d90b-81d4-48a1-a59e-d0e12998e3aa


In [31]:
client.set.default_space(space_uid)

'SUCCESS'

In [32]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt22.2-py3.10        125b6d9

In [33]:
software_spec_uid=client.software_specifications.get_uid_by_name("tensorflow_rt22.1-py3.9")
software_spec_uid

'acd9c798-6974-5d2f-a657-ce06e986df4d'

In [34]:
model_details=client.repository.store_model(model='fruit-model_new.tgz',meta_props={
client.repository.ModelMetaNames.NAME:"CNN",
client.repository.ModelMetaNames.TYPE:"tensorflow_2.7",
client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid}
                                           )
model_id=client.repository.get_model_uid(model_details)

This method is deprecated, please use get_model_id()


/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/ibm_watson_machine_learning/repository.py:1453: UserWarning: This method is deprecated, please use get_model_id()
  warn("This method is deprecated, please use get_model_id()")


In [35]:
model_id

'309b6927-2cd1-4468-a11c-250970dcfbb0'

In [36]:
client.repository.download(model_id,'my_model.tar.gz')

Successfully saved model content to file: 'my_model.tar.gz'


'/home/wsuser/work/my_model.tar.gz'